In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os

class MammographyDataset(Dataset):

	def __init__(self, csv_file, img_dir, laterality, view, transform=None):
		self.data = pd.read_csv(csv_file)
		self.img_dir = img_dir
		self.laterality = laterality
		self.view = view
		self.transform = transform

	def __len__(self):
		return len(self.data[(self.data["laterality"] == self.laterality) & (self.data["view"] == self.view)])
	

	def __getitem__(self, idx):
		row = self.data[(self.data["case_id"] == idx) & (self.data["laterality"] == self.laterality) & (self.data["view"] == self.view)]
		label = row["category"].values[0]
		img_file = f'{row["image_id"].values[0]}.jpg'
		img_path = os.path.join(self.img_dir, img_file)
		image = Image.open(img_path).convert("RGB")
        
		if self.transform:
			image = self.transform(image)
        
		return image, label


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
ds = MammographyDataset(
    csv_file='classification_tmp.csv', 
    img_dir="imgs", 
    laterality="R", 
    view="MLO", 
    transform=transform
)

dataloader = DataLoader(ds, batch_size=1, shuffle=True)

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
	model.train()
	running_loss = 0.0
	for inputs, labels in dataloader:
		optimizer.zero_grad()
		outputs = model(inputs)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()

	print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}')

print('Training complete')

0
   case_id  patient_id  study_id image_id laterality view  device_id  \
3        0           0         1  0_R_MLO          R  MLO          7   

   breast_birads  birads  category  
3              3       0         0  


(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.2275, 0.2353, 0.2275],
          [0.0000, 0.0000, 0.0000,  ..., 0.6431, 0.6353, 0.5725],
          [0.0000, 0.0000, 0.0000,  ..., 0.7569, 0.7608, 0.7137],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.3098, 0.3059, 0.3255],
          [0.0000, 0.0000, 0.0000,  ..., 0.2471, 0.2510, 0.2471],
          [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0902, 0.0980]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.2275, 0.2353, 0.2275],
          [0.0000, 0.0000, 0.0000,  ..., 0.6431, 0.6353, 0.5725],
          [0.0000, 0.0000, 0.0000,  ..., 0.7569, 0.7608, 0.7137],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.3098, 0.3059, 0.3255],
          [0.0000, 0.0000, 0.0000,  ..., 0.2471, 0.2510, 0.2471],
          [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0902, 0.0980]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.2275, 0.2353, 0.2275],
          [0.0000, 0.0000, 0.0000,  ..., 0.6431, 0.6353, 0.5725],
          [0.0000, 0.0000, 0.0000,  ...,

In [31]:
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the training images: {accuracy}%')

1
   case_id  patient_id  study_id image_id laterality view  device_id  \
7        1           1         1  1_R_MLO          R  MLO          3   

   breast_birads  birads  category  
7              1       1         1  
0
   case_id  patient_id  study_id image_id laterality view  device_id  \
3        0           0         1  0_R_MLO          R  MLO          7   

   breast_birads  birads  category  
3              3       0         0  
Accuracy of the model on the training images: 50.0%
